**SET UP THE NOTEBOOK**

In [ ]:
!pip install pdpipe

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import glob
import warnings
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import datetime as dt
import pdpipe as pdp
from typing import Tuple, List, Dict
import plotly as py
import seaborn as sns
import statistics as stat
import category_encoders as ce
import plotly.express as px
import plotly.graph_objs as go
import plotly.offline
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)
from plotly.offline import init_notebook_mode
init_notebook_mode(connected = True)
import matplotlib.pyplot as plt
%matplotlib inline
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

**CARGANDO LOS DATOS**

In [ ]:
districts = pd.read_csv('../input/learnplatform-covid19-impact-on-digital-learning/districts_info.csv')
products = pd.read_csv('../input/learnplatform-covid19-impact-on-digital-learning/products_info.csv')



# set the size of the geo bubble
def set_size(value):
    '''
    Takes the numeric value of a parameter to visualize on a map (Plotly Geo-Scatter plot)
    Returns a number to indicate the size of a bubble for a country which numeric attribute value 
    was supplied as an input
    '''
    result = np.log(1+value/100)
    if result < 0:
        result = 0.001
    return result



In [ ]:
eng_path = '../input/learnplatform-covid19-impact-on-digital-learning/engagement_data'
eng_files = glob.glob(eng_path + "/*.csv")

files = []

for file in eng_files:
    df = pd.read_csv(file, index_col = None, header = 0)
    district_id = file.split('/')[4].split('.')[0]
    df['district_id'] = district_id
    files.append(df)
    
engagement = pd.concat(files)
engagement = engagement.reset_index(drop = True)


**PROBLEMA**

La pandemia COVID-19 ha interrumpido el aprendizaje de más de 56 millones de estudiantes en los Estados Unidos. En la primavera de 2020, la mayoría de los gobiernos estatales y locales de los EE. UU. Cerraron las instituciones educativas para detener la propagación del virus. En respuesta, las escuelas y los maestros han intentado llegar a los estudiantes de forma remota a través de herramientas de aprendizaje a distancia y plataformas digitales. Hasta el día de hoy, las preocupaciones sobre la exacerbación de la brecha digital y la pérdida de aprendizaje a largo plazo entre los estudiantes más vulnerables de Estados Unidos continúan creciendo.

**OBJETIVOS**

1. Describir el panorama de la conectividad y el compromiso digital en 2020.

2. Medir el efecto de la pandemia de COVID-19 en el aprendizaje en línea y a distancia, y cómo podría evolucionar también en el futuro.
 
3. Medir el cambio de la participación de los estudiantes con los diferentes tipos de tecnología educativa durante el transcurso de la pandemia.
 
4. Medir la relación entre la participación de los estudiantes con:
 
             4.1.  Las plataformas de aprendizaje en línea y las diferentes geografías.
 
             4.2.  Contexto demográfico (por ejemplo, raza / etnia, ESL, discapacidad de aprendizaje)
  
             4.3.  Contexto de aprendizaje
 
             4.4.  Estatus socioeconómico
  
5. Determinar la correlación entre ciertas intervenciones, prácticas o políticas estatales (por ejemplo, estímulo, reapertura, moratoria de desalojo) con el aumento o la disminución de la participación en línea.

**Bases de Datos**

Los datos de participación se basan en [LearnPlatform](https://learnplatform.com/)’s Student Chrome Extension. Esta extensión recoge el evento de carga de más de 10K productos tecnológicos de educación en nuestra biblioteca de productos, incluyendo sitios web, apps, web apps, programas de software, extensiones, ebooks, hardwares, y servicios usados en instituciones académicas. 

Los datos de participación han sido agregados al nivel de escolaridad del distrito, y cada archivo representa los datos de un distrito escolar. 

El archivo de productos incluye información de las características de los mejores 372 productos con más usuarios en 2020. 

El archivo de distritos incluye información acerca del distrito escolar, incluyendo datos de: [National Center for Education Statistics (NCES)](https://nces.ed.gov/), [The Federal Communications Commission (FCC)](https://www.fcc.gov/), y [Edunomics Lab](https://edunomicslab.org/). Adicional a los archivos provistos, esperamos que uses otras fuentes de datos públicas como las que mensionamos a continuación. 

**Estructura de los archivos**

La organización del conjunto de datos se describe a continuación: ``` Root/ -engagement_data/ -1000.csv -1039.csv -... -districts_info.csv -products_info.csv -README.md ``` 

**Definicion de datos**

**Engagement data** Los datos de participación se agregan a nivel de distrito escolar y cada archivo en la carpeta `engagement_data` representa datos de un distrito escolar. 

Los cuatro digitos del nombre del archivo representados por `district_id` se pueden usar para enlazar la información del distrito en `district_info.csv`. 

El `lp_id` puede ser usado para enlazar la información en `product_info.csv`. 

**Name** Descripciòn  

**time** fecha en "YYYY-MM-DD"  

**lp_id** El identificador único del producto 

**pct_access** El porcentaje de estudiantes en el distrito tiene al menos un evento de carga de página de un producto dado en un día dado  

**engagement_index** Total de eventos de pàginas cargadas por cada cien estudiantes de un producto dado en un dìa dado  


In [ ]:
engagement.head()

In [ ]:
engagement.info()

In [ ]:
total = engagement.isnull().sum().sort_values(ascending=False)
percent = (engagement.isnull().sum()/engagement.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head(20)

In [ ]:
engagement = engagement.drop(engagement.loc[engagement['lp_id'].isnull()].index)
engagement = engagement.fillna(0.0)


In [ ]:
# cast lp_id and district_id to int, to enable merging with the products and districts info down the road
engagement["lp_id"] = engagement["lp_id"].astype(int)
engagement["district_id"] = engagement["district_id"].astype(int)
#engagements_df["time"] = pd.to_datetime(engagements_df["time"])
engagement.tail()

In [ ]:
merged_data = pd.merge(engagement, districts, on="district_id")
products.rename(columns = {'LP ID': 'lp_id'}, inplace = True)
merged_data = pd.merge(merged_data, products, on="lp_id")

In [ ]:
agg_digi_learn_df = merged_data[merged_data["Primary Essential Function"] == 'LC - Digital Learning Platforms']
agg_engagement_data = agg_digi_learn_df.groupby(["state", "time"],as_index=False)["engagement_index"].sum().reset_index()
agg_engagement_data.head(10)

In [ ]:
agg_engagement_data.tail()

In [ ]:
# kudos to https://gist.github.com/rogerallen/1583593
us_state_abbrev = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'American Samoa': 'AS',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'District of Columbia': 'DC',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Guam': 'GU',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Northern Mariana Islands':'MP',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'Puerto Rico': 'PR',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virgin Islands': 'VI',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY'
}

pipeline = pdp.PdPipeline([
    pdp.ApplyByCols('engagement_index', set_size, 'size', drop=False),
    pdp.MapColVals('state', us_state_abbrev)
])

agg_engagement_data = pipeline.apply(agg_engagement_data)

agg_engagement_data.fillna(0, inplace=True)

agg_engagement_data = agg_engagement_data.sort_values(by='time', ascending=True)
agg_engagement_data.tail()

In [ ]:
fig = px.scatter_geo(
    agg_engagement_data, locations="state", locationmode='USA-states',
    scope="usa",
    color="engagement_index", 
    size='size', hover_name="state", 
    range_color= [0, 100000], 
    projection="albers usa", animation_frame="time", 
    title='Engagement Index: LC - Digital Learning Platforms', 
    color_continuous_scale="portland")

fig.show()

In [ ]:
engagement['time'] = pd.to_datetime(engagement['time'])

**District information data** El archivo de distritos `districts_info.csv` inclye informaciòn acerca de las características del distrito escolar, incluyendo datos de [NCES](https://nces.ed.gov/) (2018-19), [FCC](https://www.fcc.gov/) (Dec 2018), y [Edunomics Lab](https://edunomicslab.org/). En este conjunto de datos, removimos informaciòn identificable del distrito escolar. también usamos una herramienta de uso libre [ARX](https://arx.deidentifier.org/) [(Prasser et al. 2020)](https://onlinelibrary.wiley.com/doi/full/10.1002/spe.2812) para transformar algunos campos y reducir el riesgo de re - identificación. Para propósotos de generalización se publican algunos datos puntuales con un rango donde el valor actual cae debajo. Adicionalmente, hay muchos datos perdidos marcados como 'NaN' indicando que los datos fueron suprimidos para garantizar el anonimato del conjunto de datos. 

**Name** Descripción 

**district_id** Identificador único del distrito escolar

**state** Elestado de residencia del distrito 

**locale** NCES clasificación local que categoriza el territorio de EEUU en cuatro tipos de áreas: Ciudad, Suburbana, Pueblo, y Rural.

**pct_black/hispanic** Porcentaje de estudiantes del distrito identificados como Negro o Hispano basado en datos de la NCES de 2018-19.

**pct_free/reduced** Porcentaje de estudiantes en los distritos elegible para almerzos gratis o a precio reducido basado en datos de 2018-19 NCES.

**county_connections_ratio** `ratio` (conexiones residenciales fijas de alta velocidad de más de 200 kbps en al menos una dirección / hogares) según los datos a nivel de condado de FCC de 477 (versión de diciembre de 2018). Para mayor informción revisar [FCC data](https://www.fcc.gov/form-477-county-data-internet-access-services). 

**pp_total_raw** Gasto total por alumno (suma de los gastos locales y federales) del proyecto de la base de datos de recursos educativos nacionales en escuelas (NERD$) de Edunomics Lab. Los datos de gastos están escuela por escuela y usamos el valor mediano para representar el gasto de un distrito escolar dado. 


In [ ]:
districts.head()

In [ ]:
missing_values_districts_count = districts.isnull().sum()
missing_values_districts_count[:]

In [ ]:
total_cells = np.product(districts.shape)
total_missing = missing_values_districts_count.sum()
percent_missing = (total_missing/total_cells) * 100
print(percent_missing)

**Product information data** El archivo de productos `products_info.csv` incluye información sobre las características de los 372 productos principales con la mayoría de los usuarios en 2020. Las categorías enumeradas en este archivo son parte de la taxonomía de productos de LearnPlatform. Nuestro equipo etiquetó los datos. Es posible que algunos productos no tengan etiquetas debido a que están duplicados, a la falta de una URL precisa u otras razones.

**Name** Descripción

**LP ID** Identificador único del producto 

**URL** Enlace Web a un producto específico 

**Product Name** Nombre de un producto específico 

**Provider/Company Name** Nombre del proveedor del producto 

**Sector(s)** Sector de educación donde se usa el producto 

**Primary Essential Function** Función básica del producto. Aquí hay dos capas de etiquetas. Los productos se etiquetan primero como una de estas tres categorías: 

**LC** = Aprendizaje y plan de estudios, 

**CM** = Gestión del salón de clases y 

**SDO** = Operaciones escolares y del distrito.

Cada una de estas categorías tiene varias subcategorías con las que se etiquetaron los productos.

**Ejemplos de otras fuentes de datos públicos relevantes** A continuación se muestran algunos ejemplos de fuentes de datos públicos relevantes (a nivel estatal): - [COVID-19 Base de datos de políticas estatales de EE. UU.]

In [ ]:
products.head()

Podemos identificar que existen celdas sin datos que fueron marcadas como NaN para indicar que los datos fueron suprimidos con el propósito de mantener el anonimato. ¿Cuántas celdas vacias hay?

In [ ]:
missing_values_products_count = products.isnull().sum()
missing_values_products_count[:]

In [ ]:
total_cells = np.product(products.shape)
total_missing = missing_values_products_count.sum()
percent_missing = (total_missing/total_cells) * 100
print(percent_missing)

La información sobre el porcentaje de estudiantes del distrito identificados como Negro o Hispano basado en datos de la NCES de 2018-19. (pct_black / hispano), porcentaje de estudiantes en los distritos elegible para almuerzos gratis o a precio reducido basado en datos de 2018-19 NCES (pct_free / reduced), conectividad en los hogares (county_connections_ratio) y el gasto por alumno (pp_total_raw) se presenta en forma de intervalos, donde "[a, b [" significa que a ≤ x <b. Todos los valores en pct_black / hispanic y pct_free / reduced tienen un intervalo de 20%, para una vista más comprensible podemos convertirlos a un solo valor con una desviación de + - 10%. La información sobre county_connections_ratio está representada por el valor más abstracto del 18% al 100% y, desafortunadamente, esta información no sirve de nada. Todos los valores de pp_total_raw tienen un intervalo de 2000, siguiendo el ejemplo de las dos columnas anteriores, convertimos los valores a un solo valor con una desviación de + - 1000.

In [ ]:
districts.dropna(inplace = True)

In [ ]:
for i in ['pct_black/hispanic', 'pct_free/reduced']:
    districts[i] = districts[i].apply(lambda x: float(x.split(',')[0][1:]) + 0.1)

districts['pp_total_raw'] = districts['pp_total_raw'].apply(lambda x: int(x.split(',')[0][1:]) + 1000)

districts.drop('county_connections_ratio', axis = 1, inplace = True)

districts.head()

In [ ]:
districts.info()

In [ ]:

state_abb = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'American Samoa': 'AS',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'District Of Columbia': 'DC',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Guam': 'GU',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Northern Mariana Islands':'MP',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'Puerto Rico': 'PR',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virgin Islands': 'VI',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY'
}

districts['state_abb'] = districts['state'].map(state_abb)

fig = go.Figure()
layout = dict(
    title_text = "Conteo de los distritos en los Estados disponibles",
    title_font = dict(
            family = "monospace",
            size = 25,
            color = "black"
            ),
    geo_scope = 'usa'
)

fig.add_trace(
    go.Choropleth(
        locations = districts['state_abb'].value_counts().to_frame().reset_index()['index'],
        zmax = 1,
        z = districts['state_abb'].value_counts().to_frame().reset_index()['state_abb'],
        locationmode = 'USA-states',
        marker_line_color = 'white',
        geo = 'geo',
        colorscale = "cividis", 
    )
)
            
fig.update_layout(layout)   
fig.show()

plt.figure(figsize = (15, 8))
sns.set_style("white")
a = sns.barplot(data = districts['state'].value_counts().reset_index(), x = 'state', y = 'index', color = '#90afc5')
plt.xticks([])
plt.yticks(fontname = 'monospace', fontsize = 14, color = '#283655')
plt.ylabel('')
plt.xlabel('')

a.spines['left'].set_linewidth(1.5)
for w in ['right', 'top', 'bottom']:
    a.spines[w].set_visible(False)
    
for p in a.patches:
    width = p.get_width()
    plt.text(0.5 + width, p.get_y() + 0.55 * p.get_height(), f'{int(width)}',
             ha = 'center', va = 'center', fontname = 'monospace', fontsize = 15, color = '#283655')

plt.show()

Los gráficos describen el número de distritos por Estado, Utah (29), Illinois (18), indiana (7), Washington (6), Missouri (5) y New York (5) corresponde al 80% del total de distritos.

En una comparación realizada por The Annie E. Casey Foundation Los estados líderes en educación en los Estados Unidos son Nueva Jersey que ocupa el primer lugar, seguido por Massachusetts y Conneticut, mientras que Oklahoma, Alaska y Nuevo México están en los últmos lugares.

Resaltemos las categorías básicas de la Función Esencial Primaria.

In [ ]:
products['Basic_category'] = 'x'
for i in range(len(products)):
    if pd.isna(products['Primary Essential Function'][i]) == False:
        products['Basic_category'][i] = products['Primary Essential Function'][i].split('-')[0][:-1]
        
products.head()

In [ ]:
fig = px.pie(districts['locale'].value_counts().reset_index().rename(columns = {'locale': 'count'}), values = 'count', names = 'index', width = 700, height = 700)

fig.update_traces(textposition = 'inside', 
                  textinfo = 'percent + label', 
                  hole = 0.7, 
                  marker = dict(colors = ['#90afc5','#336b87','#2a3132','#763626'], line = dict(color = 'white', width = 2)))

fig.update_layout(annotations = [dict(text = ' Conteo de los distritos <br>en cada tipo <br>de área', 
                                      x = 0.5, y = 0.5, font_size = 26, showarrow = False, 
                                      font_family = 'monospace',
                                      font_color = '#283655')],
                  showlegend = False)
                  
fig.show()

El Centro Nacional para Estadística Educativa (NCES) define cuatro grupos de localidad de acuerdo a su proximidad con la zona rural y el número de habitantes. El gráfico muestra que el 59% de los distritos se encuentran en zonas suburbanas, es decir, que se encuentran fuera del perimetro de la ciudad prinipal de un área metropolitana, seguido porla zona rural 17%, ciudad 13,6% y pueblo 10,2%.

In [ ]:
colors = ['#90afc5', '#336b87', '#763626']

fig = plt.figure(figsize = (15, 11))
for i in range(len(districts.columns.tolist()[3:6])):
    plt.subplot(2, 2, i+1)
    sns.set_style("white")
    plt.title(districts.columns.tolist()[3:6][i], size = 20, fontname = 'monospace', color = colors[i])
    a = sns.kdeplot(districts[districts.columns.tolist()[3:6][i]], color = colors[i], shade = True, alpha = 0.9, linewidth = 1.5, edgecolor = 'black')
    plt.ylabel('')
    plt.xlabel('')
    plt.xticks(fontname = 'monospace')
    plt.yticks([])
    for j in ['right', 'left', 'top']:
        a.spines[j].set_visible(False)
        a.spines['bottom'].set_linewidth(1.2)
        
fig.tight_layout(h_pad = 3)

plt.figtext(0.07, 1.05, 'Distribución de las características del distrito escolar', fontsize = 30, fontname = 'monospace', color = '#283655')

plt.figtext(0.84, 0.37, 'Conclusión', fontsize = 25, fontname = 'monospace', color = '#283655')

plt.figtext(0.98, 0.15, '''El número promedio de estudiantes que se identificaron
como negros o hispanos es el 23,2%. El valor más común es el 10%.

El número promedio de estudiantes elegibles 
para recibir servicios gratuitos o de precio reducido
el almuerzo es del 38%. Los valores más comunes son ~ 30% y ~ 50%.

Gasto total por alumno (suma de los
gastos) es de 11 205 $. 
El valor más común es 9 000 \ $.''', fontsize = 13, fontname = 'monospace', color = '#283655', ha = 'right')

plt.show()


In [ ]:
dist_area_group = districts.groupby('locale').agg({'pct_black/hispanic': 'mean', 'pct_free/reduced': 'mean', 'pp_total_raw': 'mean'}).reset_index()

colors = ['#90afc5', '#336b87', '#763626']

fig = plt.figure(figsize = (13, 12))
for i in range(len(dist_area_group.columns.tolist()[1:])):
    plt.subplot(2, 2, i+1)
    sns.set_style("white")
    plt.title(dist_area_group.columns.tolist()[1:][i], size = 20, fontname = 'monospace', y = 1.09, color = colors[i])
    plt.grid(color = 'gray', linestyle = ':', axis = 'y', zorder = 0,  dashes = (1,7))
    a = sns.barplot(data = dist_area_group, x = 'locale', y = dist_area_group.columns.tolist()[1:][i], color = colors[i])
    plt.ylabel('')
    plt.xlabel('')
    plt.xticks(fontname = 'monospace', size = 14)
    plt.yticks([])
    
    for j in ['right', 'top', 'left']:
        a.spines[j].set_visible(False)
    for j in ['bottom']:
        a.spines[j].set_linewidth(1.4)
      
    if i < 2:
        for p in a.patches:
            height = p.get_height()
            a.annotate(f'{int(height*100)} %', (p.get_x() + p.get_width() / 2, p.get_height()-0.03), 
                   ha = 'center', va = 'center', 
                   size = 18,
                   xytext = (0, 5), 
                   textcoords = 'offset points',
                   color = 'white',
                   fontname = 'monospace')
    else:
        for p in a.patches:
            height = p.get_height()
            a.annotate(f'{int(height)} $', (p.get_x() + p.get_width() / 2, p.get_height()-1000), 
                   ha = 'center', va = 'center', 
                   size = 18,
                   xytext = (0, 5), 
                   textcoords = 'offset points',
                   color = 'white',
                   fontname = 'monospace')
            
plt.figtext(0.07, 1.05, 'Características de los distritos escolares por localidad', fontsize = 30, fontname = 'monospace', color = '#283655')

plt.figtext(0.83, 0.34, 'Conclusión', fontsize = 25, fontname = 'monospace', color = '#283655')

plt.figtext(0.99, 0.15, '''El mayor recuento de estudiantes que se identificaron
como negros o hispanos en las grandes ciudades.
El recuento más pequeño se encuentra 
en los suburbios y áreas rurales.

En ciudades y pueblos, la mitad de 
los estudiantes son elegibles.
para almuerzo gratis o a precio reducido.

Los gastos totales más altos por alumno 
se encuentran en la zona rural.''', fontsize = 13, fontname = 'monospace', color = '#283655', ha = 'right')
        
fig.tight_layout(pad = 3)

plt.show()

**PRODUCTOS**

In [ ]:
products.info()

In [ ]:
# Very important! The data is filled in incorrectly, which is why maaany people has an error, 
#because of which, for example, Adobe Inc company has 3, not 4 products.
for i in [' ', '.']:
    for k in range(len(products)):
        if pd.isna(products['Provider/Company Name'][k]) == False:
            products['Provider/Company Name'][k] = products['Provider/Company Name'][k].strip(i)

# Let's fix the long name
products['Provider/Company Name'] = np.where(products['Provider/Company Name'] == 'Savvas Learning Company | Formerly Pearson K12 Learning', 'Savvas Learning Company', products['Provider/Company Name'])

plt.figure(figsize = (15, 8))
sns.set_style("white")
plt.title('TOP-15 de compañías proveedoras de aprendizaje', size = 35, x = 0.48, y = 1.06, fontname = 'monospace', color = '#283655')
a = sns.barplot(data = products['Provider/Company Name'].value_counts().reset_index().head(15), x = 'Provider/Company Name', y = 'index', color = '#90afc5')
plt.xticks([])
plt.yticks(fontname = 'monospace', fontsize = 14, color = '#283655')
plt.ylabel('')
plt.xlabel('')

a.spines['left'].set_linewidth(1.5)
for w in ['right', 'top', 'bottom']:
    a.spines[w].set_visible(False)
    
for p in a.patches:
    width = p.get_width()
    plt.text(0.5 + width, p.get_y() + 0.55 * p.get_height(), f'{int(width)}',
             ha = 'center', va = 'center', fontname = 'monospace', fontsize = 15, color = '#283655')

plt.show()

Google LLC es el proveedor líder de aprendizaje en línea con 30 productos seguido de Microsoft con 6 productos.

In [ ]:
fig = px.pie(products['Sector(s)'].value_counts().reset_index().rename(columns = {'Sector(s)': 'count'}).head(15), values = 'count', names = 'index', width = 700, height = 700)

fig.update_traces(textposition = 'inside', 
                  textinfo = 'percent + label', 
                  hole = 0.7, 
                  marker = dict(colors = ['#90afc5','#336b87','#2a3132','#763626', 'a43820'], line = dict(color = 'white', width = 2)))

fig.update_layout(annotations = [dict(text = 'Sector de educación <br>donde se usa el producto', 
                                      x = 0.5, y = 0.5, font_size = 26, showarrow = False, 
                                      font_family = 'monospace',
                                      font_color = '#283655')],
                  showlegend = False)
                  
fig.show()

Los productos de educación en línea son utilizados en un 42% por Prek-12, en un 32,7% por Prek-12, Higher Ed; Corpoerate, un 18% por Prek-12, Higher Ed.

In [ ]:
plt.figure(figsize = (12, 12))
sns.set_style("white")
plt.title('Conteo de productos por subcategoría', size = 35, x = 0.2, y = 1.06, fontname = 'monospace', color = '#283655')
a = sns.barplot(data = products['Primary Essential Function'].value_counts().reset_index(), x = 'Primary Essential Function', y = 'index', color = '#90afc5')
plt.xticks([])
plt.yticks(fontname = 'monospace', fontsize = 10, color = '#283655')
plt.ylabel('')
plt.xlabel('')

a.spines['left'].set_linewidth(1.5)
for w in ['right', 'top', 'bottom']:
    a.spines[w].set_visible(False)
    
for p in a.patches:
    width = p.get_width()
    plt.text(1 + width, p.get_y() + 0.55 * p.get_height(), f'{int(width)}',
             ha = 'center', va = 'center', fontname = 'monospace', fontsize = 11, color = '#283655')

plt.show()
##########
fig = px.pie(products.query("Basic_category != 'x'")['Basic_category'].value_counts().reset_index().rename(columns = {'Basic_category': 'count'}), values = 'count', names = 'index', width = 700, height = 700)

fig.update_traces(textposition = 'inside', 
                  textinfo = 'percent + label', 
                  hole = 0.7, 
                  marker = dict(colors = ['#90afc5','#336b87','#2a3132','#763626'], line = dict(color = 'white', width = 2)))

fig.update_layout(annotations = [dict(text = 'Conteo de productos <br>por categoría', 
                                      x = 0.5, y = 0.5, font_size = 26, showarrow = False, 
                                      font_family = 'monospace',
                                      font_color = '#283655')],
                  showlegend = False)
                  
fig.show()

Existen 74 subcategorias de productos dirigidos al aprendizae en línea que representa un 77,3%, un 9,66% corresponde a la gestión del salón en clases y un 8,52% a operaciones escolares y de distrito. 

**ACTIVIDADES DE LOS ESTUDIANTES**

In [ ]:
merged_data = pd.merge(products, engagement, left_on = 'lp_id', right_on = 'lp_id')
merged_data['district_id'] = merged_data['district_id'].astype('int64')
merged_data = pd.merge(merged_data, districts, on = 'district_id')
merged_data.drop(['URL', 'lp_id', 'state_abb'], axis = 1, inplace = True)
merged_data.head()

In [ ]:
st_acсess = merged_data.groupby(['state', 'time']).agg({'pct_access': 'mean'}).reset_index()
st_eng = merged_data.groupby(['state', 'time']).agg({'engagement_index': 'mean'}).reset_index()
loc_acсess = merged_data.groupby(['locale', 'time']).agg({'pct_access': 'mean'}).reset_index()
loc_eng = merged_data.groupby(['locale', 'time']).agg({'engagement_index': 'mean'}).reset_index()
cat_acсess = merged_data.groupby(['Basic_category', 'time']).agg({'pct_access': 'mean'}).reset_index()
cat_eng = merged_data.groupby(['Basic_category', 'time']).agg({'engagement_index': 'mean'}).reset_index()

for i in [st_acсess, st_eng, loc_acсess, loc_eng, cat_acсess, cat_eng]:
    i['day_of_week'] = i['time'].dt.dayofweek
    
st_acсess.head(3)

In [ ]:
fig = px.line(st_acсess, x="time", y="pct_access", color="state", line_group="state")

fig.update_layout(plot_bgcolor = 'white', title = 'Dinámicas de pct_access de todos los productos por estado', 
                  title_font_family = 'monospace', title_font_color = '#221f1f', title_font_size = 20, title_x = 0.5)
fig.update_xaxes(showline = True, linecolor = '#f5f2f2', linewidth = 2, tickfont_family = 'monospace', tickfont_color = '#221f1f', tickfont_size = 12)
fig.update_yaxes(showline = True, linecolor = '#f5f2f2', 
                 showgrid = True, gridwidth = 1, gridcolor = '#f5f2f2',
                 linewidth = 2, tickfont_family = 'monospace', tickfont_color = '#221f1f', tickfont_size = 12)

fig.add_vline(x = '2020-03-11', line_width = 3, line_color="red")

fig.add_annotation(
        x='2020-03-11',
        y=2.7,
        text="WHO has declared Covid-19 a pandemic",
        showarrow=True,
        font=dict(
            family="monospace",
            size=11,
            color="black"
            ),
        arrowhead=2,
        arrowsize=1,
        arrowwidth=2,
        arrowcolor="#636363",
        ax= 130,
        ay=1
        )

fig.add_vrect(x0="2020-06-01", x1="2020-08-31", fillcolor="yellow", opacity=0.25, line_width=0)

fig.add_annotation(
        x='2020-07-15',
        y=2.25,
        text="Vacaciones",
        showarrow=False,
        font=dict(
            family="monospace",
            size=11,
            color="black"
            )
        )

fig.update_traces(line_width=1)

fig.show()

In [ ]:
fig = px.line(st_eng, x="time", y="engagement_index", color="state", line_group="state")

fig.update_layout(plot_bgcolor = 'white', title = 'Índice de dinámica de participación de todos los productos por estados', 
                  title_font_family = 'monospace', title_font_color = '#221f1f', title_font_size = 20, title_x = 0.5)
fig.update_xaxes(showline = True, linecolor = '#f5f2f2', linewidth = 2, tickfont_family = 'monospace', tickfont_color = '#221f1f', tickfont_size = 12)
fig.update_yaxes(showline = True, linecolor = '#f5f2f2', 
                 showgrid = True, gridwidth = 1, gridcolor = '#f5f2f2',
                 linewidth = 2, tickfont_family = 'monospace', tickfont_color = '#221f1f', tickfont_size = 12)

fig.add_vline(x = '2020-03-11', line_width = 3, line_color="red")

fig.add_annotation(
        x='2020-03-11',
        y=1150,
        text="WHO has declared Covid-19 a pandemic",
        showarrow=True,
        font=dict(
            family="monospace",
            size=11,
            color="black"
            ),
        arrowhead=2,
        arrowsize=1,
        arrowwidth=2,
        arrowcolor="#636363",
        ax= 130,
        ay=1
        )

fig.add_vrect(x0="2020-06-01", x1="2020-08-31", fillcolor="yellow", opacity=0.25, line_width=0)

fig.add_annotation(
        x='2020-07-15',
        y=900,
        text="Summer holidays",
        showarrow=False,
        font=dict(
            family="monospace",
            size=11,
            color="black"
            )
        )

fig.update_traces(line_width=1)

fig.show()

In [ ]:
months_map = {1:"January",2:"February",3:"March",4:"April",
              5:"May",6:"June",7:"July",8:"August",9:"September",
              10:"October",11:"November",12:"December"}

for i in [st_acсess, st_eng]:
    i['state_abb'] = i['state'].map(state_abb)
    i['month'] = i.time.dt.month.map(months_map)

    fig = px.choropleth(data_frame = i.groupby(['state', 'state_abb', 'month']).agg({i.columns[2]: 'mean'}).reset_index(), locations = "state_abb", locationmode = "USA-states",
                    color = i.groupby(['state', 'state_abb', 'month']).agg({i.columns[2]: 'mean'}).reset_index()[i.groupby(['state', 'state_abb', 'month']).agg({i.columns[2]: 'mean'}).reset_index().columns[3]], scope = "usa",
                    color_continuous_scale = "cividis", animation_frame = "month", hover_name = "state")
    
    fig.update_layout(title_text = f'Dinámica mensual de {i.columns[2]}', title_font = dict(family = "monospace",size = 25,color = "black")) 
    
    fig.show()

**Cambios en los indicadores de actividad promedio de los estudiantes en los días escolares 1 y 2 semanas después del anuncio de la pandemia en todos los estados.**

In [ ]:
cov_imp = pd.DataFrame(st_acсess['state'].unique().tolist()).rename(columns = {0: 'state'})

# We have no information about Texas during the start of pandemic
cov_imp = cov_imp.query("state != 'Texas'").reset_index()
cov_imp.drop('index', axis = 1, inplace = True)

for i in ['mean_access', '1w_acess_change%', '2w_acess_change%', 'mean_eng', '1w_eng_change%', '2w_eng_change%']:
    cov_imp[i] = 0.0

states = cov_imp['state'].unique().tolist()

for i in states:
    cov_imp['mean_access'][states.index(i)] = round(st_acсess.query("time >= '2020-03-09' & time <= '2020-03-13' & state == @i")['pct_access'].mean(), 2)
    cov_imp['1w_acess_change%'][states.index(i)] = round((st_acсess.query("time >= '2020-03-16' & time <= '2020-03-20' & state == @i")['pct_access'].mean() / cov_imp['mean_access'][states.index(i)] - 1) * 100, 1)
    cov_imp['2w_acess_change%'][states.index(i)] = round((st_acсess.query("time >= '2020-03-23' & time <= '2020-03-27' & state == @i")['pct_access'].mean() / st_acсess.query("time >= '2020-03-16' & time <= '2020-03-20' & state == @i")['pct_access'].mean() - 1) * 100, 1)
    cov_imp['mean_eng'][states.index(i)] = round(st_eng.query("time >= '2020-03-09' & time <= '2020-03-13' & state == @i")['engagement_index'].mean(), 1)
    cov_imp['1w_eng_change%'][states.index(i)] = round((st_eng.query("time >= '2020-03-16' & time <= '2020-03-20' & state == @i")['engagement_index'].mean() / cov_imp['mean_eng'][states.index(i)] - 1) * 100, 1)
    cov_imp['2w_eng_change%'][states.index(i)] = round((st_eng.query("time >= '2020-03-23' & time <= '2020-03-27' & state == @i")['engagement_index'].mean() / st_eng.query("time >= '2020-03-16' & time <= '2020-03-20' & state == @i")['engagement_index'].mean() - 1) * 100, 1)

def color_values(val):
    color = 'red' if val < 0 else 'green'
    return 'color: %s' % color

slice_ = ['1w_acess_change%', '2w_acess_change%', '1w_eng_change%', '2w_eng_change%']
slice_2 = ['mean_access', '1w_acess_change%', '2w_acess_change%']
slice_3 = ['mean_eng', '1w_eng_change%', '2w_eng_change%']
cov_imp.style.applymap(color_values, subset = slice_).set_precision(1).set_properties(**{'background-color': '#fafafa'}, subset=slice_2).set_properties(**{'background-color': '#f7f7f7'}, subset=slice_3)


El impacto del COVID - 19 en el aprendizaje digital durante la primera y segunda semana después de decalrada la pandemia está representdo en color rojo aquellos que disminuyeron y en color verde aquellos que aumentaron su participación o mostraron mayor interes en el aprendizaje en línea.

In [ ]:
fig = px.line(loc_acсess, x="time", y="pct_access", color="locale", line_group="locale")

fig.update_layout(plot_bgcolor = 'white', title = 'Dinámica de pct_access de todos los productos por localidad', 
                  title_font_family = 'monospace', title_font_color = '#221f1f', title_font_size = 20, title_x = 0.5)
fig.update_xaxes(showline = True, linecolor = '#f5f2f2', linewidth = 2, tickfont_family = 'monospace', tickfont_color = '#221f1f', tickfont_size = 12)
fig.update_yaxes(showline = True, linecolor = '#f5f2f2', 
                 showgrid = True, gridwidth = 1, gridcolor = '#f5f2f2',
                 linewidth = 2, tickfont_family = 'monospace', tickfont_color = '#221f1f', tickfont_size = 12)

fig.add_vline(x = '2020-03-11', line_width = 3, line_color="red")

fig.add_annotation(
        x='2020-03-11',
        y=2,
        text="WHO has declared Covid-19 a pandemic",
        showarrow=True,
        font=dict(
            family="monospace",
            size=11,
            color="black"
            ),
        arrowhead=2,
        arrowsize=1,
        arrowwidth=2,
        arrowcolor="#636363",
        ax= 130,
        ay=1
        )

fig.add_vrect(x0="2020-06-01", x1="2020-08-31", fillcolor="yellow", opacity=0.25, line_width=0)

fig.add_annotation(
        x='2020-07-15',
        y=1.75,
        text="Summer holidays",
        showarrow=False,
        font=dict(
            family="monospace",
            size=11,
            color="black"
            )
        )

fig.update_traces(line_width=1)

fig.show()

In [ ]:
fig = px.line(loc_eng, x="time", y="engagement_index", color="locale", line_group="locale")

fig.update_layout(plot_bgcolor = 'white', title = 'Índice de dinámica de participación de todos los productos por ubicación', 
                  title_font_family = 'monospace', title_font_color = '#221f1f', title_font_size = 20, title_x = 0.5)
fig.update_xaxes(showline = True, linecolor = '#f5f2f2', linewidth = 2, tickfont_family = 'monospace', tickfont_color = '#221f1f', tickfont_size = 12)
fig.update_yaxes(showline = True, linecolor = '#f5f2f2', 
                 showgrid = True, gridwidth = 1, gridcolor = '#f5f2f2',
                 linewidth = 2, tickfont_family = 'monospace', tickfont_color = '#221f1f', tickfont_size = 12)

fig.add_vline(x = '2020-03-11', line_width = 3, line_color="red")

fig.add_annotation(
        x='2020-03-11',
        y=700,
        text="WHO has declared Covid-19 a pandemic",
        showarrow=True,
        font=dict(
            family="monospace",
            size=11,
            color="black"
            ),
        arrowhead=2,
        arrowsize=1,
        arrowwidth=2,
        arrowcolor="#636363",
        ax= 130,
        ay=1
        )

fig.add_vrect(x0="2020-06-01", x1="2020-08-31", fillcolor="yellow", opacity=0.25, line_width=0)

fig.add_annotation(
        x='2020-07-15',
        y=610,
        text="Summer holidays",
        showarrow=False,
        font=dict(
            family="monospace",
            size=11,
            color="black"
            )
        )

fig.update_traces(line_width=1)

fig.show()

**Cambios en los indicadores de actividad de los estudiantes promedio en los días escolares 1 y 2 semanas después del anuncio de la pandemia en todos los lugares.**

In [ ]:
cov_imp2 = pd.DataFrame(loc_acсess['locale'].unique().tolist()).rename(columns = {0: 'locale'})

for i in ['mean_access', '1w_acess_change%', '2w_acess_change%', 'mean_eng', '1w_eng_change%', '2w_eng_change%']:
    cov_imp2[i] = 0.0

locales = cov_imp2['locale'].unique().tolist()

for i in locales:
    cov_imp2['mean_access'][locales.index(i)] = round(loc_acсess.query("time >= '2020-03-09' & time <= '2020-03-13' & locale == @i")['pct_access'].mean(), 2)
    cov_imp2['1w_acess_change%'][locales.index(i)] = round((loc_acсess.query("time >= '2020-03-16' & time <= '2020-03-20' & locale == @i")['pct_access'].mean() / cov_imp2['mean_access'][locales.index(i)] - 1) * 100, 1)
    cov_imp2['2w_acess_change%'][locales.index(i)] = round((loc_acсess.query("time >= '2020-03-23' & time <= '2020-03-27' & locale == @i")['pct_access'].mean() / loc_acсess.query("time >= '2020-03-16' & time <= '2020-03-20' & locale == @i")['pct_access'].mean() - 1) * 100, 1)
    cov_imp2['mean_eng'][locales.index(i)] = round(loc_eng.query("time >= '2020-03-09' & time <= '2020-03-13' & locale == @i")['engagement_index'].mean(), 1)
    cov_imp2['1w_eng_change%'][locales.index(i)] = round((loc_eng.query("time >= '2020-03-16' & time <= '2020-03-20' & locale == @i")['engagement_index'].mean() / cov_imp2['mean_eng'][locales.index(i)] - 1) * 100, 1)
    cov_imp2['2w_eng_change%'][locales.index(i)] = round((loc_eng.query("time >= '2020-03-23' & time <= '2020-03-27' & locale == @i")['engagement_index'].mean() / loc_eng.query("time >= '2020-03-16' & time <= '2020-03-20' & locale == @i")['engagement_index'].mean() - 1) * 100, 1)

cov_imp2.style.applymap(color_values, subset = slice_).set_precision(1).set_properties(**{'background-color': '#fafafa'}, subset=slice_2).set_properties(**{'background-color': '#f7f7f7'}, subset=slice_3)


Un analisis de la variación por localidad muestra que la participacion de los alumnos en la ciudad disminuyó en un 64,5% mientras que en los sectores suburbanos aumento en 7,4%. Estos resultados son relevantes al momento de nalizar el factor de la distancia en la educación. 

In [ ]:
fig = px.line(cat_acсess.query("Basic_category != 'x'"), x="time", y="pct_access", color="Basic_category", line_group="Basic_category")

fig.update_layout(plot_bgcolor = 'white', title = 'Dinámica de pct_access de todos los productos por categoría', 
                  title_font_family = 'monospace', title_font_color = '#221f1f', title_font_size = 20, title_x = 0.5)
fig.update_xaxes(showline = True, linecolor = '#f5f2f2', linewidth = 2, tickfont_family = 'monospace', tickfont_color = '#221f1f', tickfont_size = 12)
fig.update_yaxes(showline = True, linecolor = '#f5f2f2', 
                 showgrid = True, gridwidth = 1, gridcolor = '#f5f2f2',
                 linewidth = 2, tickfont_family = 'monospace', tickfont_color = '#221f1f', tickfont_size = 12)

fig.add_vline(x = '2020-03-11', line_width = 3, line_color="red")

fig.add_annotation(
        x='2020-03-11',
        y=4.5,
        text="WHO has declared Covid-19 a pandemic",
        showarrow=True,
        font=dict(
            family="monospace",
            size=11,
            color="black"
            ),
        arrowhead=2,
        arrowsize=1,
        arrowwidth=2,
        arrowcolor="#636363",
        ax= 130,
        ay=1
        )

fig.add_vrect(x0="2020-06-01", x1="2020-08-31", fillcolor="yellow", opacity=0.25, line_width=0)

fig.add_annotation(
        x='2020-07-15',
        y=3.8,
        text="Summer holidays",
        showarrow=False,
        font=dict(
            family="monospace",
            size=11,
            color="black"
            )
        )

fig.update_traces(line_width=1)

fig.show()

In [ ]:
fig = px.line(cat_eng.query("Basic_category != 'x'"), x="time", y="engagement_index", color="Basic_category", line_group="Basic_category")

fig.update_layout(plot_bgcolor = 'white', title = 'Índice de dinámica de participación de todos los productos por categoría de producto', 
                  title_font_family = 'monospace', title_font_color = '#221f1f', title_font_size = 20, title_x = 0.5)
fig.update_xaxes(showline = True, linecolor = '#f5f2f2', linewidth = 2, tickfont_family = 'monospace', tickfont_color = '#221f1f', tickfont_size = 12)
fig.update_yaxes(showline = True, linecolor = '#f5f2f2', 
                 showgrid = True, gridwidth = 1, gridcolor = '#f5f2f2',
                 linewidth = 2, tickfont_family = 'monospace', tickfont_color = '#221f1f', tickfont_size = 12)

fig.add_vline(x = '2020-03-11', line_width = 3, line_color="red")

fig.add_annotation(
        x='2020-03-11',
        y=1900,
        text="WHO has declared Covid-19 a pandemic",
        showarrow=True,
        font=dict(
            family="monospace",
            size=11,
            color="black"
            ),
        arrowhead=2,
        arrowsize=1,
        arrowwidth=2,
        arrowcolor="#636363",
        ax= 130,
        ay=1
        )

fig.add_vrect(x0="2020-06-01", x1="2020-08-31", fillcolor="yellow", opacity=0.25, line_width=0)

fig.add_annotation(
        x='2020-07-15',
        y=1600,
        text="Summer holidays",
        showarrow=False,
        font=dict(
            family="monospace",
            size=11,
            color="black"
            )
        )

fig.update_traces(line_width=1)

fig.show()

**Cambios en los indicadores de actividad de los estudiantes promedio en los días escolares 1 y 2 semanas después del anuncio de la pandemia en cada categoría de producto.**

In [ ]:
cov_imp3 = pd.DataFrame(cat_eng.query("Basic_category != 'x'")['Basic_category'].unique().tolist()).rename(columns = {0: 'Basic_category'})

for i in ['mean_access', '1w_acess_change%', '2w_acess_change%', 'mean_eng', '1w_eng_change%', '2w_eng_change%']:
    cov_imp3[i] = 0.0

categories = cov_imp3['Basic_category'].unique().tolist()

for i in categories:
    cov_imp3['mean_access'][categories.index(i)] = round(cat_acсess.query("time >= '2020-03-09' & time <= '2020-03-13' & Basic_category == @i")['pct_access'].mean(), 2)
    cov_imp3['1w_acess_change%'][categories.index(i)] = round((cat_acсess.query("time >= '2020-03-16' & time <= '2020-03-20' & Basic_category == @i")['pct_access'].mean() / cov_imp3['mean_access'][categories.index(i)] - 1) * 100, 1)
    cov_imp3['2w_acess_change%'][categories.index(i)] = round((cat_acсess.query("time >= '2020-03-23' & time <= '2020-03-27' & Basic_category == @i")['pct_access'].mean() / cat_acсess.query("time >= '2020-03-16' & time <= '2020-03-20' & Basic_category == @i")['pct_access'].mean() - 1) * 100, 1)
    cov_imp3['mean_eng'][categories.index(i)] = round(cat_eng.query("time >= '2020-03-09' & time <= '2020-03-13' & Basic_category == @i")['engagement_index'].mean(), 1)
    cov_imp3['1w_eng_change%'][categories.index(i)] = round((cat_eng.query("time >= '2020-03-16' & time <= '2020-03-20' & Basic_category == @i")['engagement_index'].mean() / cov_imp3['mean_eng'][categories.index(i)] - 1) * 100, 1)
    cov_imp3['2w_eng_change%'][categories.index(i)] = round((cat_eng.query("time >= '2020-03-23' & time <= '2020-03-27' & Basic_category == @i")['engagement_index'].mean() / cat_eng.query("time >= '2020-03-16' & time <= '2020-03-20' & Basic_category == @i")['engagement_index'].mean() - 1) * 100, 1)

cov_imp3.style.applymap(color_values, subset = slice_).set_precision(1).set_properties(**{'background-color': '#fafafa'}, subset=slice_2).set_properties(**{'background-color': '#f7f7f7'}, subset=slice_3)

La participación promedio de los estudiantes luego de declarada la pandemia respecto a las categorías de productos disminuyó para las de aprendizaje y plan de estudios y las operaciones escolares y de distrito, sin embargo aumentó para las categorías de gestión del salón de clases, es decir, los profesores.

**Correlación de datos**

In [ ]:
# features correlations 
plt.figure(figsize=(8,6))
corr_matrix = engagement.corr(method ='pearson')
ax = sns.heatmap(corr_matrix, cmap="YlGnBu", annot=True)

Existe corelación en un 75% del porcentaje de acceso y la participación de los estudiantes.